Verification of 3.2.1 Stochiometric Surfaces

start by creating surfaces with flourite structure

In [1]:
from ase.io import read, write
from ase.build import surface, make_supercell, sort

structure = read("geometry_file\CeO2_fluorite.cif")

In [16]:
from ase.io import read, write
from ase.build import surface, make_supercell, sort

structure = read("geometry_file\CeO2_fluorite.cif")
slab = surface(structure, (1,1,1), layers=2, vacuum=10.0)
write('test111slab.cif', slab, format='cif')


In [ ]:
# (111) surface
slab = surface(structure, (1,1,1), layers=16, vacuum=10.0)
slab.center(vacuum=15.0, axis=2)
# remove the top layers
top_layer_z = max(slab.positions[:, 2])
slab = slab[[atom.index for atom in slab if atom.position[2] < top_layer_z]]
# remove 5 bottom layers
for i in range(5):
    bottom_layer_z = min(slab.positions[:, 2])
    slab = slab[[atom.index for atom in slab if atom.position[2] > bottom_layer_z]]

supercell = make_supercell(slab,[[4,0,0],[0,4,0],[0,0,1]])
#write('111slab.cif', slab, format='cif')
#write('111supercell.cif', supercell, format='cif')

In [14]:
# (110) surface
slab = surface(structure, (1,1,0), layers=20, vacuum=10.0)
slab.center(vacuum=20.0, axis=2)

for i in range(2):
    top_layer_z = max(slab.positions[:, 2])
    slab = slab[[atom.index for atom in slab if atom.position[2] < top_layer_z]]

supercell = make_supercell(slab,[[4,0,0],[0,4,0],[0,0,1]])

x_front = 0.1
x_back = 28.9
supercell = supercell[[atom.index for atom in supercell if atom.position[0] < x_back and atom.position[0] > x_front]]

#write('110slab.cif', slab, format='cif')
#write('110supercell.cif', supercell, format='cif')
write('new110supercell.cif', supercell, format='cif')

In [2]:
#new110slab
from ase.io import read, write
from ase.build import surface, make_supercell

slab = read("geometry_file/110slab.cif")
supercell = make_supercell(slab,[[2,0,0],[0,1,0],[0,0,1]])
bottom_z = 75.0
front_x = 3.86
supercell = supercell[[atom.index for atom in supercell if atom.position[2] > bottom_z]]
supercell = supercell[[atom.index for atom in supercell if atom.position[0] > front_x]]
write('new110slab.cif',supercell,format='cif')


In [ ]:
# (100) surface
slab = surface(structure, (1,0,0), layers=9, vacuum=10.0)
slab.center(vacuum=15.0, axis=2)

for i in range(2):
    top_layer_z = max(slab.positions[:, 2])
    slab = slab[[atom.index for atom in slab if atom.position[2] < top_layer_z]]

supercell = make_supercell(slab,[[4,0,0],[0,4,0],[0,0,1]])
#write('geometry_file/100slab.cif', slab, format='cif')
#write('geometry_file/100supercell.cif', supercell, format='cif')

the (111) and (110) surface shown in VESTA look same with the figure given in paper, the (100) surface however, hasn't been reconstructed and thus looks not good.

we will continue to calculate the bulk energy and slab energy

In [42]:
from lammps import lammps
import numpy as np

conv1 = 4184 # kcal/mol to J
conv2 = 1e-20 # A^2 to m^2
lmp_bulk = lammps()
lmp_bulk.file("ceo2fluorite.lmp")
E_bulk = lmp_bulk.get_thermo("pe")
print(E_bulk)

-2464.422720390472


In [43]:
nBulk_111 = 224
A_111 = 828.4211067 # A^2
lmp111 = lammps()
lmp111.file("ceo2slab111.lmp")
E_slab = lmp111.get_thermo("pe")
E_surf_111 = (E_slab - nBulk_111 * E_bulk) * conv1 / (2 * A_111 * conv2) 
print(E_surf_111)

3.3567318432791593e+25


with unit conversion to J/m^2 E_surf_111 = -3.3535632570698284e+25

E_slab = -353219.429526086

In [37]:
nBulk_110 = 312
A_110 = 676.4022963 # A^2
lmp110 = lammps()
lmp110.file("ceo2slab110.lmp")
E_slab = lmp110.get_thermo("pe")
E_surf_110 = (E_slab - nBulk_110 * E_bulk) * conv1 / (2 * A_110 * conv2)
print(E_surf_110)

-4.376385522399618e+23


and E_surf_110 = -7.530549496249226 for 110 surface.

with unit conversion E_surf_110 = -3.910978540820418e+25

In [11]:
def add_charges_to_atoms(input_file, output_file):
    # Define the charges for each atom type
    atom_charges = {
        1: 4.0,  # Ce
        2: -2.0  # O
    }

    # Read the file and store its contents
    with open(input_file, "r") as file:
        lines = file.readlines()

    # Find the line number where atom information starts
    atom_info_line = None
    for i, line in enumerate(lines):
        if line.strip().startswith("Atoms"):
            atom_info_line = i + 1
            break

    # If the line number is found, add charges to each atom
    if atom_info_line is not None:
        # Iterate through atom lines and add charges
        for i in range(atom_info_line, len(lines)):
            atom_line = lines[i].split()
            if len(atom_line) < 2:
                continue  # Skip lines that do not have enough elements
            atom_index = atom_line[0]
            atom_type = int(atom_line[1])
            charge = atom_charges.get(atom_type)
            if charge is not None:
                # Modify the line to include charge
                lines[i] = f"         {atom_line[0]}         {atom_line[1]}   {charge}        {atom_line[2]}       {atom_line[3]}       {atom_line[4]}\n"

    # Write the modified contents back to the output file
    with open(output_file, "w") as file:
        file.writelines(lines)

Manually run this function only once to add charges for slab.lmp

In [15]:
#add_charges_to_atoms("geometry_file/111supercell.lmp","geometry_file/111supercell.lmp")
#add_charges_to_atoms("geometry_file/110supercell.lmp","geometry_file/110supercell.lmp")
#add_charges_to_atoms("geometry_file/100slab.lmp","geometry_file/100slab.lmp")
#add_charges_to_atoms("geometry_file/111slab.lmp","geometry_file/e111slab.lmp")
#add_charges_to_atoms("geometry_file/new110slab.lmp","geometry_file/new110slab.lmp")
add_charges_to_atoms("new110supercell.lmp","new110supercell.lmp")